## Transfer Learning using MNIST data
To illustrate the power and concept of transfer learning, we will train a CNN on just the digits 5,6,7,8,9.  Then we will train just the last layer(s) of the network on the digits 0,1,2,3,4 and see how well the features learned on 5-9 help with classifying 0-4.

Adapted from https://github.com/fchollet/keras/blob/master/examples/mnist_transfer_cnn.py

In [1]:
from __future__ import print_function

import datetime
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

Using TensorFlow backend.


In [2]:
# Used to help some of the timing functions
now = datetime.datetime.now

In [3]:
# Set some parameters
batch_size = 128
num_classes = 5
epochs = 5

In [4]:
# Set some more parameters
img_rows, img_cols = 28, 28
filters = 32
pool_size = 2
kernel_size = 3

In [5]:
## This just handles some variability in how the input data is loaded
if K.image_data_format() == 'channels_first':
    input_shape = (1, img_rows, img_cols)
else:
    input_shape = (img_rows, img_cols, 1)

In [6]:
## To simplify things, use a function to include all the training steps
## As input, function takes a model, training set, test set, and the number of classes
## Inside the model object will be the state about which layers we are freezing and which we are training

def train_model(model, train, test, num_classes):
    x_train = train[0].reshape((train[0].shape[0],) + input_shape)
    x_test = test[0].reshape((test[0].shape[0],) + input_shape)
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    print('x_train shape:', x_train.shape)
    print(x_train.shape[0], 'train samples')
    print(x_test.shape[0], 'test samples')

    # convert class vectors to binary class matrices
    y_train = keras.utils.to_categorical(train[1], num_classes)
    y_test = keras.utils.to_categorical(test[1], num_classes)

    model.compile(loss='categorical_crossentropy',
                  optimizer='adadelta',
                  metrics=['accuracy'])

    t = now()
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_data=(x_test, y_test))
    print('Training time: %s' % (now() - t))

    score = model.evaluate(x_test, y_test, verbose=0)
    print('Test score:', score[0])
    print('Test accuracy:', score[1])

In [7]:
# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# create two datasets: one with digits below 5 and one with 5 and above
x_train_lt5 = x_train[y_train < 5]
y_train_lt5 = y_train[y_train < 5]
x_test_lt5 = x_test[y_test < 5]
y_test_lt5 = y_test[y_test < 5]

x_train_gte5 = x_train[y_train >= 5]
y_train_gte5 = y_train[y_train >= 5] - 5
x_test_gte5 = x_test[y_test >= 5]
y_test_gte5 = y_test[y_test >= 5] - 5

In [8]:
# Define the "feature" layers.  These are the early layers that we expect will "transfer"
# to a new problem.  We will freeze these layers during the fine-tuning process

feature_layers = [
    Conv2D(filters, kernel_size,
           padding='valid',
           input_shape=input_shape),
    Activation('relu'),
    Conv2D(filters, kernel_size),
    Activation('relu'),
    MaxPooling2D(pool_size=pool_size),
    Dropout(0.25),
    Flatten(),
]

In [9]:
# Define the "classification" layers.  These are the later layers that predict the specific classes from the features
# learned by the feature layers.  This is the part of the model that needs to be re-trained for a new problem

classification_layers = [
    Dense(128),
    Activation('relu'),
    Dropout(0.5),
    Dense(num_classes),
    Activation('softmax')
]

In [10]:
# We create our model by combining the two sets of layers as follows
model = Sequential(feature_layers + classification_layers)

In [11]:
# Let's take a look
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
activation_1 (Activation)    (None, 26, 26, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 24, 24, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 4608)              0         
__________

In [12]:
# Now, let's train our model on the digits 5,6,7,8,9

train_model(model,
            (x_train_gte5, y_train_gte5),
            (x_test_gte5, y_test_gte5), num_classes)

x_train shape: (29404, 28, 28, 1)
29404 train samples
4861 test samples
Train on 29404 samples, validate on 4861 samples
Epoch 1/5
29404/29404 [==============================] - 150s - loss: 0.2978 - acc: 0.9005 - val_loss: 0.0951 - val_acc: 0.9687
Epoch 2/5
29404/29404 [==============================] - 153s - loss: 0.1062 - acc: 0.9664 - val_loss: 0.0543 - val_acc: 0.9819
Epoch 3/5
29404/29404 [==============================] - 154s - loss: 0.0743 - acc: 0.9763 - val_loss: 0.0464 - val_acc: 0.9844
Epoch 4/5
29404/29404 [==============================] - 151s - loss: 0.0588 - acc: 0.9813 - val_loss: 0.0331 - val_acc: 0.9881
Epoch 5/5
29404/29404 [==============================] - 152s - loss: 0.0502 - acc: 0.9849 - val_loss: 0.0301 - val_acc: 0.9887
Training time: 0:12:45.003994
Test score: 0.0301443396063
Test accuracy: 0.988685455668


### Freezing Layers
Keras allows layers to be "frozen" during the training process.  That is, some layers would have their weights updated during the training process, while others would not.  This is a core part of transfer learning, the ability to train just the last one or several layers.

Note also, that a lot of the training time is spent "back-propagating" the gradients back to the first layer.  Therefore, if we only need to compute the gradients back a small number of layers, the training time is much quicker per iteration.  This is in addition to the savings gained by being able to train on a smaller data set.

In [13]:
# Freeze only the feature layers
for l in feature_layers:
    l.trainable = False

Observe below the differences between the number of *total params*, *trainable params*, and *non-trainable params*.

In [14]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
activation_1 (Activation)    (None, 26, 26, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 24, 24, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 4608)              0         
__________

In [15]:
train_model(model,
            (x_train_lt5, y_train_lt5),
            (x_test_lt5, y_test_lt5), num_classes)

x_train shape: (30596, 28, 28, 1)
30596 train samples
5139 test samples
Train on 30596 samples, validate on 5139 samples
Epoch 1/5
30596/30596 [==============================] - 65s - loss: 0.2560 - acc: 0.9241 - val_loss: 0.0353 - val_acc: 0.9887
Epoch 2/5
30596/30596 [==============================] - 66s - loss: 0.0695 - acc: 0.9799 - val_loss: 0.0215 - val_acc: 0.9922
Epoch 3/5
30596/30596 [==============================] - 67s - loss: 0.0492 - acc: 0.9858 - val_loss: 0.0161 - val_acc: 0.9944
Epoch 4/5
30596/30596 [==============================] - 60s - loss: 0.0386 - acc: 0.9891 - val_loss: 0.0138 - val_acc: 0.9949
Epoch 5/5
30596/30596 [==============================] - 50s - loss: 0.0333 - acc: 0.9903 - val_loss: 0.0116 - val_acc: 0.9959
Training time: 0:05:10.441598
Test score: 0.011593080794
Test accuracy: 0.995913601868


Note that after a single epoch, we are already achieving results on classifying 0-4 that are comparable to those achieved on 5-9 after 5 full epochs.  This despite the fact the we are only "fine-tuning" the last layer of the network, and all the early layers have never seen what the digits 0-4 look like.

Also, note that even though nearly all (590K/600K) of the *parameters* were trainable, the training time per epoch was still much reduced.  This is because the unfrozen part of the network was very shallow, making backpropagation faster. 

## Train on Digits 0-4

In [16]:
for l in feature_layers:
    l.trainable = True
    
train_model(model,
            (x_train_lt5, y_train_lt5),
            (x_test_lt5, y_test_lt5), num_classes)

x_train shape: (30596, 28, 28, 1)
30596 train samples
5139 test samples
Train on 30596 samples, validate on 5139 samples
Epoch 1/5
30596/30596 [==============================] - 145s - loss: 0.0311 - acc: 0.9913 - val_loss: 0.0100 - val_acc: 0.9963
Epoch 2/5
30596/30596 [==============================] - 142s - loss: 0.0288 - acc: 0.9917 - val_loss: 0.0088 - val_acc: 0.9967
Epoch 3/5
30596/30596 [==============================] - 148s - loss: 0.0235 - acc: 0.9931 - val_loss: 0.0076 - val_acc: 0.9971
Epoch 4/5
30596/30596 [==============================] - 137s - loss: 0.0214 - acc: 0.9933 - val_loss: 0.0072 - val_acc: 0.9977
Epoch 5/5
30596/30596 [==============================] - 137s - loss: 0.0180 - acc: 0.9945 - val_loss: 0.0063 - val_acc: 0.9981
Training time: 0:11:53.008394
Test score: 0.00629821651318
Test accuracy: 0.998054096128


In [17]:
for l in feature_layers:
    l.trainable = False
    
train_model(model,
            (x_train_gte5, y_train_gte5),
            (x_test_gte5, y_test_gte5), num_classes)

x_train shape: (29404, 28, 28, 1)
29404 train samples
4861 test samples
Train on 29404 samples, validate on 4861 samples
Epoch 1/5
29404/29404 [==============================] - 57s - loss: 0.1959 - acc: 0.9462 - val_loss: 0.0393 - val_acc: 0.9881
Epoch 2/5
29404/29404 [==============================] - 60s - loss: 0.0645 - acc: 0.9791 - val_loss: 0.0306 - val_acc: 0.9907
Epoch 3/5
29404/29404 [==============================] - 60s - loss: 0.0521 - acc: 0.9840 - val_loss: 0.0256 - val_acc: 0.9924
Epoch 4/5
29404/29404 [==============================] - 60s - loss: 0.0451 - acc: 0.9858 - val_loss: 0.0253 - val_acc: 0.9922
Epoch 5/5
29404/29404 [==============================] - 50s - loss: 0.0413 - acc: 0.9873 - val_loss: 0.0246 - val_acc: 0.9912
Training time: 0:04:50.628174
Test score: 0.0246485681481
Test accuracy: 0.991154083412
